# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.<br>

My general notes:<br>
Have in mind, that we work on a multi-class, multi-label text classification which assigns to each message sample a set of category target labels. The messages are short and an imbalanced data distribution exists. The dataset has 19634 data points with 32 different target categories.

During the disaster messages processing, the English text is tokenized, lower cased, lemmatized and the contractions are expanded. Additionally, e.g. spaces, punctuation and English stop words are removed.


### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [60]:
#
# import libraries
#

# download necessary NLTK data
#%pip install nltk
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import random as rn
import numpy as np
import pandas as pd
import string
import pickle
from sqlalchemy import create_engine

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#%pip install bs4
from bs4 import BeautifulSoup

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# warnings status to show
import warnings
warnings.warn("once")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ilona\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ilona\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilona\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: once


Make the code reproducible ...

In [3]:
FIXED_SEED = 42

# The below is necessary for starting NumPy generated random numbers in a well-defined initial state.
np.random.seed(FIXED_SEED)

# The below is necessary for starting core Python generated random numbers in a well-defined state.
rn.seed(FIXED_SEED)

In [4]:
# load data from database
try:
    engine = create_engine('sqlite:///Disaster_Messages_engine.db')
    df = pd.read_sql_table('Messages_Categories_table', engine)
    
    # success
    print("The dataset has {} data points with {} variables each.".format(*df.shape))
except:
    print("The database 'Disaster_Messages_engine.db' could not be loaded. No ML pipeline activities possible.")

The dataset has 19634 data points with 40 variables each.


In [5]:
df.head()

,message,original,genre,lang_code,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,en,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,en,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,en,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,en,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,en,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [6]:
# create input (X) and output (y) samples, we know that related is always one ...
# as input we have to take care about the messages
# the categories are the targets of the multi-class, multi-label classification
X = df['message']
y = df[df.columns[4:]]
TARGET_NAMES = y.columns

In [7]:
print("X datatype: {}".format(type(X)))
print("y datatype: {}".format(type(y)))

X datatype: <class 'pandas.core.series.Series'>
y datatype: <class 'pandas.core.frame.DataFrame'>


In [8]:
X.head(2)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [9]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [10]:
y.iloc[0:5,:].values

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]], dtype=int64)

In [11]:
for group in y.columns:
    print("'{}' includes {} x value 1.".format(group, y[group].sum()))

'related' includes 19634 x value 1.
'request' includes 4374 x value 1.
'offer' includes 117 x value 1.
'aid_related' includes 10729 x value 1.
'medical_help' includes 2066 x value 1.
'medical_products' includes 1297 x value 1.
'search_and_rescue' includes 718 x value 1.
'security' includes 467 x value 1.
'military' includes 857 x value 1.
'child_alone' includes 0 x value 1.
'water' includes 1650 x value 1.
'food' includes 2885 x value 1.
'shelter' includes 2281 x value 1.
'clothing' includes 401 x value 1.
'money' includes 598 x value 1.
'missing_people' includes 297 x value 1.
'refugees' includes 872 x value 1.
'death' includes 1187 x value 1.
'other_aid' includes 3392 x value 1.
'infrastructure_related' includes 1688 x value 1.
'transport' includes 1197 x value 1.
'buildings' includes 1313 x value 1.
'electricity' includes 528 x value 1.
'tools' includes 158 x value 1.
'hospitals' includes 283 x value 1.
'shops' includes 118 x value 1.
'aid_centers' includes 308 x value 1.
'other_inf

### 2. Write a tokenization function to process your text data

During EPL pipeline activities we realised that there are messages which are not useful (e.g. 'nonsense' character sequences, html characters) and there are probably web links included. We have to deal with this in the tokenize() function.

In [12]:
CONTRACTION_MAP = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [13]:
# function from Dipanjan's repository:
# https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/bonus%\
# 20content/nlp%20proven%20approach/NLP%20Strategy%20I%20-%20Processing%20and%20Understanding%20Text.ipynb

def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    
    return expanded_text

In [14]:
stop_words = set(stopwords.words('english'))
#stop_words.remove('no')
#stop_words.remove('not')

def tokenize(text):
    # have in mind that we use this for a web app adding new messages;
    # if still html, xml or other undefined parts in the existing messages:
    # first remove such metatext from English messages
    # see: https://docs.python.org/3.7/library/codecs.html#encodings-and-unicode
    # "To be able to detect the endianness of a UTF-16 or UTF-32 byte sequence,
    # there’s the so called BOM (“Byte Order Mark”). [...]
    # In UTF-8, the use of the BOM is discouraged and should generally be avoided."
    # specific ones are e.g. notepad signatures from Microsoft as part of the messages which should be avoided;
    # other undefined characters have the coding of the 'replacement character' unicode u"\ufffd"
    soup = BeautifulSoup(text, 'html')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'       
    detected_urls = re.findall(url_regex, bom_removed)
    for url in detected_urls:
        text = bom_removed.replace(url, "urlplaceholder")
        
    # change the negation wordings like don't to do not, won't to will not 
    # or other contractions like I'd to I would, I'll to I will etc. via dictionary
    text = expand_contractions(text, CONTRACTION_MAP)

    # remove punctuation [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]
    text = text.translate(str.maketrans('','', string.punctuation))
    # remove numbers
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    # during ETL pipeline we have reduced the dataset on English messages ('en' language coding,
    # but there can be some wrong codings
    tokens = word_tokenize(letters_only, language='english')
    lemmatizer = WordNetLemmatizer()  # for the lexical correctly found word stem (root)

    clean_tokens = []
    for tok in tokens:
        # use only lower cases, remove leading and ending spaces
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        # remember: there have been nonsense sentences, so, now some strings could be empty
        # toDo: what is the correct length number to use now? Small ones are probably no relevant words ...
        # remove English stop words
        if (len(clean_tok) > 1) & (clean_tok not in stop_words):
            clean_tokens.append(clean_tok)

    return clean_tokens

In [15]:
# example for unit test to remove punctuation [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]
example_str = 'This [is an] example? {of} string. with.? some &punctuation &signs!!??!!'
result = example_str.translate(str.maketrans('','', string.punctuation))
print(result)
# output shall be: This is an example of string with some punctuation signs

This is an example of string with some punctuation signs


In [16]:
# test tokenize
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 

Storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'tent', 'water', 'silo', 'thank'] 

I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )
['croixdesbouquets', 'health', 'issue', 'worker', 'santo', 'area', 'croixdesbouquets'] 

There's nothing to eat and water, we starving and t

### 3. Build a machine learning pipeline
Notes:
- Regarding the class default parameters, for this Python implementation scikit-learn version 0.21.2 is used.
- We use np.random.seed() too beside of random_state/random_seed parameters ([reason](https://stackoverflow.com/questions/47923258/random-seed-on-svm-sklearn-produces-different-results))
- For the pipeline workflow a `FeatureUnion`instance concatenates results of multiple transformer objects

Remember, we are dealing with an imbalanced dataset, therefore not all models can be used. A machine learning classifier could be more biased towards the majority class, causing bad classification of the minority class. Therefore we have to take care.

This machine pipeline should take in the `message` column as input and output classification results on the other remaining 31 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

According scikit-learn [documentation](https://scikit-learn.org/stable/modules/multiclass.html) we can choose only specific classifier using this meta-estimator. We start with `RandomForestClassier`.<br>

In [17]:
pipeline = Pipeline([
        ('features', FeatureUnion([ 
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
            ]))
            
        ])),
    
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, class_weight='balanced',
                                                             n_jobs=1, random_state=FIXED_SEED)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
# shuffle is by default set on True,
# usage of stratify param leads to stratify split technique for this imbalanced dataset,
# having both would be a StratifiedShuffleSplit algorithm in the background,
# but
# stratify=y leads to a ValueError: The least populated class in y has only 1 member, which is too few.
# The minimum number of groups for any class cannot be less than 2.
# ToDo: clarify why => must be: stratify=y.iloc[:,1]
# Therefore I added randomised resampling to reduce the probability of getting ValueError's during model fit()
# and after clarification comment them out

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, stratify=y.iloc[:,1],
                                                    test_size=0.2, random_state=FIXED_SEED)

In [19]:
X_train.shape

(15707,)

In [20]:
y_train.shape

(15707, 36)

In [21]:
print("X_train datatype: {}".format(type(X_train)))
print("y_train datatype: {}".format(type(y_train)))

X_train datatype: <class 'numpy.ndarray'>
y_train datatype: <class 'numpy.ndarray'>


In [22]:
for i in range(y_train.shape[1]):
    print("{}. numpy.ndarray element is: {}".format(i, y_train[i]))
    print(set(y_train[i]))

0. numpy.ndarray element is: [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1]
{0, 1}
1. numpy.ndarray element is: [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
{0, 1}
2. numpy.ndarray element is: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{0, 1}
3. numpy.ndarray element is: [1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
{0, 1}
4. numpy.ndarray element is: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{0, 1}
5. numpy.ndarray element is: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{0, 1}
6. numpy.ndarray element is: [1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{0, 1}
7. numpy.ndarray element is: [1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{0, 1}
8. numpy.ndarray element is: [1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
{0, 1}
9. numpy.n

Now, we train the pipeline ...

In [23]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

And calculate the model prediction ...

In [24]:
y_rfc_pred = pipeline.predict(X_test)

### 5. Test your model
For evaluation:<br>
Report accuracy score, f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each, where:

TP = TruePositive; FP = FalsePositive; TN = TrueNegative; FN = FalseNegative.

**Accuracy Score** is a classification score. It is the number of correct predictions made divided by the total number of predictions made. In a multilabel classification task it computes subset accuracy. 
  
Furthermore, beside accuracy, we add additional metrics to compare the model performance having an originally imbalanced dataset. Accuracy would focus too much on the majority classes. Because of this overfitting of the majority classes, its value would be too good and therefore misleading.

**Precision** quantifies the binary precision. In other words, a measure of a classifiers exactness. It is a ratio of true positives (messages correctly classified to their categories)) to all positives (all messages classified to categories, irrespective of whether that was the correct classification), in other words it is the ratio of

TP / (TP + FP)

**Recall** tells us what proportion of messages that actually were classified to specific categories were classified by us as this categories. Means, a measure of a classifiers completeness. It is a ratio of true positives to all the correctly category classified messages that were actually disaster messages, in other words it is the ratio of

TP / (TP + FN)

A model's ability to precisely predict those that are correctly categoriesed disaster messages is more important than the model's ability to recall those individuals. 

We can use **F-beta score** as a metric that considers both precision and recall. According scikit-learn, the F-beta score is the weighted harmonic mean of precision and recall, reaching its optimal value at 1 and its worst value at 0. F – Measure is nothing but the harmonic mean of Precision and Recall.

Fβ=(1 + β2)  (precision⋅recall / ((β2⋅precision) + recall))

In particular, when β=0.5, more emphasis is placed on precision. And when β=1.0 recall and precision are equally important.

According scikit-learn: "The **F1 score** ... reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)

In the multi-class and multi-label case, this is the average of the F1 score of each class with weighting depending on the average parameter."

From scikit-learn documentation for the classification report:<br>
The classification_report() function returns an additional value: **Support** - the number of occurrences of each label in y_true.<br>
The reported averages include macro average (averaging the unweighted mean per label), weighted average (averaging the support-weighted mean per label), sample average (only for multilabel classification) and micro average (averaging the total true positives, false negatives and false positives) it is only shown for multi-label or multi-class with a subset of classes because it is accuracy otherwise.

Note: Having the imbalanced dataset in mind, Cohen's Kappa and Confusion Matrix are not possible because this is a multi-label classification task.

In [25]:
def display_results(target_names, y_test, y_pred, cv=None, parameters=None):
   
    # text summary of the overall accuracy, precision, recall, F1 score for each class   
    print("\nFirst: overall accuracy score: {:5f}".format(accuracy_score(y_test, y_pred)))

    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    # shows F1_score, precision and recall
    class_report = classification_report(y_test, y_pred, target_names=target_names)
    print("Classification Report for each target class:\n", class_report)
    
    if cv != None:
        print("\n\n---- Best Parameters: ----\n")
        print("Best score: {:3f}".format(cv.best_score_))
        print("Best estimators parameters set:")
        best_parameters = cv.best_estimator_.get_params()
        for param_name in sorted(parameters.keys()):
            print("\t {}: {}".format(param_name, best_parameters[param_name]))  

In [26]:
display_results(TARGET_NAMES, y_test, y_rfc_pred, None, None)


First: overall accuracy score: 0.070283
Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.56      0.39      0.46       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.58      0.70      0.63      2146
          medical_help       0.14      0.00      0.01       386
      medical_products       0.18      0.01      0.02       253
     search_and_rescue       0.20      0.01      0.01       142
              security       0.00      0.00      0.00       103
              military       0.00      0.00      0.00       179
           child_alone       0.00      0.00      0.00         0
                 water       0.10      0.00      0.01       332
                  food       0.21      0.01      0.01       553
               shelter       0.06      0.00      0.00       456
              cl

C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Such kind of behaviour has been expected because having an imbalanced dataset and in the output vectors for each message, most of the target label values are set to 0 - only few are set to 1.<br>
The accuracy metric is not an appropriate measure to evaluate model performance of such kind of dataset. It could classify all instances as part of the majority class and classifies the minority class targets as noise. It is not able to evaluate the model performance of a multi-class dataset with multi-output vectors.<br>
Additionally in this classification report, often the metrics are not reliable because of being set to 0.0 according calculation rules. If values are available, precision is in general higher than recall, in other words, we have a high rate of false negatives (all items wrongly classified as not being part of the specific target class). Therefore we start to improve the model by using cross-validated hyperparameters.

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [28]:
# specify parameters for grid search
rfc_param_grid = {
    'features__text_pipeline__vect__ngram_range': [(1,2), (1,3)],
    'clf__estimator__n_estimators': [1000, 2000],
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__class_weight': ['balanced']
}

# create grid search object
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
# cv not higher than 5 buckets, training needs days with cv=10 if e.g. amazon AWS EC2 service is not available
# n_jobs set to 1 because cloud service throws TerminatedWorkerError if > 1
# for scoring and refit see: https://stackoverflow.com/questions/57591311/combination-of-gridsearchcvs-refit-and-scorer-unclear
#scoring = {'f1': make_scorer(f1_score, average="samples"), 'Accuracy': make_scorer(accuracy_score)}
grid_cv = GridSearchCV(pipeline, param_grid=rfc_param_grid, n_jobs=1, cv=5,
                       return_train_score=True, verbose=2)# scoring = scoring, refit='f1', return_train_score=True, verbose=2)

### 7. Test your model
Show the accuracy, precision, recall and F-score of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
# model = cv
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2), total=18.6min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 26.8min remaining:    0.0s


[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2), total=18.8min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2), total=19.0min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 2), total=17.

[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 3), total=47.5min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 3) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 3), total=46.1min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 3) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1, 3), total=46.7min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=20, clf__estimator__n_estimators=1000, features__text_pipeline__vect__ngram_range=(1,

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 2537.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [30]:
y_rfc_pred2 = grid_cv.predict(X_test)
y_rfc_pred2

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1]], dtype=int64)

In [31]:
print("CV results:")
sorted(grid_cv.cv_results_.keys())

CV results:


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_clf__estimator__class_weight',
 'param_clf__estimator__max_depth',
 'param_clf__estimator__n_estimators',
 'param_features__text_pipeline__vect__ngram_range',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [32]:
for param_name, param_value in zip(grid_cv.cv_results_.keys(), grid_cv.cv_results_.values()):
    print(param_name, "=", param_value, "\n")

mean_fit_time = [ 951.01102476 1289.35329661 1897.77585025 2530.09342275 1985.3813417
 2492.7219656  3751.74686284 5012.73669758] 

std_fit_time = [31.64974161 14.4293965  40.56762986 20.34711407 41.96932656 45.73615276
 84.01539479 41.12148111] 

mean_score_time = [168.82307229 290.16513877 338.12631183 570.52026534 198.09174027
 295.95815959 402.81810784 605.61343603] 

std_score_time = [4.76827371 3.21221783 1.79543797 6.13089396 8.51926294 3.68539973
 3.82734014 5.46997596] 

param_clf__estimator__class_weight = ['balanced' 'balanced' 'balanced' 'balanced' 'balanced' 'balanced'
 'balanced' 'balanced'] 

param_clf__estimator__max_depth = [10 10 10 10 20 20 20 20] 

param_clf__estimator__n_estimators = [1000 1000 2000 2000 1000 1000 2000 2000] 

param_features__text_pipeline__vect__ngram_range = [(1, 2) (1, 3) (1, 2) (1, 3) (1, 2) (1, 3) (1, 2) (1, 3)] 

params = [{'clf__estimator__class_weight': 'balanced', 'clf__estimator__max_depth': 10, 'clf__estimator__n_estimators': 1000, 'feat

In [33]:
type(grid_cv.best_estimator_)

sklearn.pipeline.Pipeline

In [34]:
print("Evaluation results for the 5 buckets cross validation tuned 'RandomForestClassifier' estimator:")
display_results(TARGET_NAMES, y_test, y_rfc_pred2, grid_cv, rfc_param_grid)

Evaluation results for the 5 buckets cross validation tuned 'RandomForestClassifier' estimator:

First: overall accuracy score: 0.053731
Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.50      0.76      0.60       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.58      0.65      0.61      2146
          medical_help       0.10      0.01      0.01       386
      medical_products       0.17      0.00      0.01       253
     search_and_rescue       0.00      0.00      0.00       142
              security       0.00      0.00      0.00       103
              military       0.00      0.00      0.00       179
           child_alone       0.00      0.00      0.00         0
                 water       0.14      0.08      0.10       332
                  food       0.25      0.41     

C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The tuned evaluation result of the RandomForestClassifier with tuned hyperparameters is not better, there are still much categories set to 0.0 and having a low best score.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

**First**, we try out other machine learning algorithms which are tuned by cross validation to compare their prediction results. Other estimator models for the requested `MultiOutputClassifier` are:
- `KNeighborsClassifier` with its default parameters: (n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)
- `RadiusNeighborsClassifier` with its default parameters: (radius=1.0, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, outlier_label=None, metric_params=None, n_jobs=None, **kwargs)

As stated in the scikit-learn [documentation](https://scikit-learn.org/stable/modules/neighbors.html#classification) "scikit-learn implements two different nearest neighbors classifiers: <i>KNeighborsClassifier</i> implements learning based on the nearest neighbors of each query point, where is an integer value specified by the user.<br> <i>RadiusNeighborsClassifier</i> implements learning based on the number of neighbors within a fixed radius of each training point, where is a floating-point value specified by the user."
    
**Second**, because it is an imbalanced dataset we could do a balancing before classification. The categority classes with low numbers of observations are outnumbered. So, the dataset is highly skewed. To create a balanced dataset several strategies exists:
- Undersampling the majority classes
- Oversampling the minority classes
- Combining over- and under-sampling
- Create ensemble balanced sets

But have in mind, that minority class oversampling could result in overfitting problems doing it before cross-validation. We would link the information of validation data to our training dataset which is forbidden.

Note:<br>
Doing balancing activities the specific scikit package 'imbalanced-learn' is imported.<br>
For combining the strategies we implement a naive random oversampling of the minority classes.<br>
For undersampling the package can be used as well to create the pipeline with `PipelineImb`. The pipeline itself includes the class `RandomUnderSampler()` directly before the MultiOutputClassifier to equalize the number of samples in all the classes before the training.

Using such package throws the following ValueError: 'Imbalanced-learn currently supports binary, multiclass and binarized encoded multiclasss targets. Multilabel and multioutput targets are not supported.' So, the associated package classes do not support the multi-target classification with multiple outputs as we need for our project. Therefore this coding is removed after such experiment.

According the [paper](https://arxiv.org/ftp/arxiv/papers/1810/1810.11612.pdf) <i>Handling Imbalanced Dataset in Multi-label Text Categorization using Bagging and Adaptive Boosting</i> of 27 October 2018 from Genta Indra Winata and Masayu Leylia Khodra, regarding new data, it is more appropriate to balance the dataset on the algorithm level instead of the data level to avoid overfitting. The algorithm "approach modifies algorithm by adjusting weight or cost of various classes."

If the usage of the mentioned specific library is not possible for our task, what could we do instead?<br>
Another option is a `feature-selection` approach which can be done after the feature extraction of the `TfidfVectorizer`, which is creating [feature vectors](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer).

Scikit-learn offers the package [feature decomposition](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition). With its help a subsampling is added:
- For the sparse matrix delivered from the `TfidfVectorizer` instance we use 2500 most frequent text features, each feature token shall appear at least 5 times and n-gram wording during grid search hyperparameter tuning.
- Feature relationship of the sparse matrix is handled with `TruncatedSVD` for latent semantic analysis (LSA). There, a component relationship parameter is evaluated via grid search hyperparameter tuning. Afterwards we have to normalise again.

In [82]:
def build_model(model_type, params):
    ''' 
    input:
    model_type - the estimator model used for the MultiOutputClassifier
    params - the estimator model parameter grid used for the GridSearchCV 
    ''' 
    
    # TfidfVectorizer, by default: use_idf=True, norm=’l2’
    # TruncatedSVD: for SLA n_components of 100 is recommended, but it is stated:
    # Desired dimensionality of output data. Must be strictly less than the number of features.
    # We have 36 features and want to have 30.
    #
    # For RandomizedSearchCV: we have 8 parameters => n_iter=8
    pipeline2 = Pipeline([
        ('features', FeatureUnion([            
            ('text_pipeline', Pipeline([
                ('tfidf', TfidfVectorizer(tokenizer=tokenize, sublinear_tf=True,                                   
                                          max_features=2500, min_df=5)),
                ('best', TruncatedSVD(n_components=30, random_state=FIXED_SEED)),
                ('normalizer', Normalizer(copy=False))
            ]))           
        ])),

        ('clf', MultiOutputClassifier(model_type))
    ])
    
    # the higher the verbose number the more information is thrown
    cv = GridSearchCV(pipeline2, param_grid=params, return_train_score=True, n_jobs=1, cv=5, verbose=2)
    #cv = RandomizedSearchCV(pipeline2, param_distributions=params, n_jobs=1, cv=5, n_iter=8,
    #                        verbose=2, random_state=FIXED_SEED)
    
    return cv

We try this new pipeline first with the 2 other mentioned classifiers and afterwards with an additionally tuned RandomForestClassifier.

In [36]:
# create param grids for the models
knn_param_grid  = {
    'features__text_pipeline__tfidf__ngram_range': [(1, 2), (1,3)],
    'clf__estimator__weights': ['uniform', 'distance']
}

rn_param_grid  = {
    'features__text_pipeline__tfidf__ngram_range': [(1, 2), (1,3)],
    'clf__estimator__weights': ['uniform', 'distance']
}

In [74]:
print("\n----- KNeighborsClassifier with feature selection -----")
print("Build best model: ...")
cv_knn_model = build_model(KNeighborsClassifier(n_neighbors=10, n_jobs=1), knn_param_grid)
print("Train model: ...")
cv_knn_model.fit(X_train, y_train)


----- KNeighborsClassifier with feature selection -----
Build best model: ...
Train model: ...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total= 6.1min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.1min remaining:    0.0s


[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=12.4min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total= 6.0min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total= 5.8min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total= 5.7min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3), total= 5.9min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3) 
[CV]  clf__estimator__wei

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 121.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('tfidf',
                                                                                         TfidfVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.float64'>,

In [75]:
y_knn_pred = cv_knn_model.predict(X_test)

In [39]:
type(cv_knn_model.estimator)

sklearn.pipeline.Pipeline

In [40]:
type(cv_knn_model.estimator['features']) 

sklearn.pipeline.FeatureUnion

In [41]:
type(cv_knn_model.estimator['features'].get_params()['transformer_list'][0])

tuple

In [42]:
type(cv_knn_model.estimator['features'].get_params()['transformer_list'][0][1])

sklearn.pipeline.Pipeline

In [76]:
cv_knn_model.estimator['features'].get_params()['transformer_list'][0][1]

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=2500,
                                 min_df=5, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=True,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000004F491A8840>,
                                 use_idf=True, vocabulary=None)),
                ('best',
                 TruncatedSVD(algorithm='randomized', n_components=30, n_iter=5,
                             

In [77]:
print("Best score: %0.3f" % cv_knn_model.best_score_)
print("Best parameters set:")
best_parameters = cv_knn_model.best_estimator_.get_params()
for param_name in sorted(knn_param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.112
Best parameters set:
	clf__estimator__weights: 'uniform'
	features__text_pipeline__tfidf__ngram_range: (1, 2)


In [78]:
print("\nModel evaluation on tuned KNeighborsClassifier ...")
display_results(TARGET_NAMES, y_test, y_knn_pred, cv_knn_model, knn_param_grid)


Model evaluation on tuned KNeighborsClassifier ...

First: overall accuracy score: 0.115355


C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.53      0.25      0.34       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.58      0.51      0.54      2146
          medical_help       0.00      0.00      0.00       386
      medical_products       0.75      0.01      0.02       253
     search_and_rescue       0.00      0.00      0.00       142
              security       0.00      0.00      0.00       103
              military       0.00      0.00      0.00       179
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       332
                  food       0.48      0.02      0.03       553
               shelter       0.00      0.00      0.00       456
              clothing       0.00      0.00      0.00    

In [79]:
print("\n----- RadiusNeighborsClassifier with feature selection -----")
print("Build best model: ...")
cv_rn_model = build_model(RadiusNeighborsClassifier(n_jobs=1), rn_param_grid)
print("Train model: ...")
cv_rn_model.fit(X_train, y_train)


----- RadiusNeighborsClassifier with feature selection -----
Build best model: ...
Train model: ...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=16.0min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.0min remaining:    0.0s


[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=15.7min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=16.5min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=16.0min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 2), total=16.4min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3) 
[CV]  clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3), total=17.0min
[CV] clf__estimator__weights=uniform, features__text_pipeline__tfidf__ngram_range=(1, 3) 
[CV]  clf__estimator__wei

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 333.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('tfidf',
                                                                                         TfidfVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.float64'>,

In [80]:
y_rn_pred = cv_rn_model.predict(X_test)

In [81]:
print("\nModel evaluation on tuned RadiusNeighborsClassifier ...")
display_results(TARGET_NAMES, y_test, y_rn_pred, cv_rn_model, rn_param_grid)


Model evaluation on tuned RadiusNeighborsClassifier ...

First: overall accuracy score: 0.023428


C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.57      0.15      0.24       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.57      0.90      0.69      2146
          medical_help       0.00      0.00      0.00       386
      medical_products       0.00      0.00      0.00       253
     search_and_rescue       0.00      0.00      0.00       142
              security       0.00      0.00      0.00       103
              military       0.00      0.00      0.00       179
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       332
                  food       0.00      0.00      0.00       553
               shelter       0.00      0.00      0.00       456
              clothing       0.00      0.00      0.00    

Regarding the evaluation results of both new models, it is worse and not acceptable comparing the single target features. Nevertheless, the KNeighborsClassifier has a better best score value compared to the RandomForestClassifier without decomposition. Would the feature selection and decomposition improve the RandomForestClassifier?

In [83]:
better_rfc_param_grid = {
    'features__text_pipeline__tfidf__ngram_range': [(1,2), (1,3)],
    'clf__estimator__n_estimators': [2000, 2500],
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__class_weight': ['balanced']
}

In [ ]:
print("\n----- RandomForestClassifier with feature selection and better param grid -----")
print("Build best model: ...")
cv_better_rfc_model = build_model(RandomForestClassifier(n_jobs=1, random_state=FIXED_SEED),
                                  better_rfc_param_grid)
print("Train model: ...")
cv_better_rfc_model.fit(X_train, y_train)


----- RandomForestClassifier with feature selection and better param grid -----
Build best model: ...
Train model: ...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2), total=504.3min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 573.6min remaining:    0.0s


[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2), total=520.4min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2), total=391.1min
[CV] clf__estimator__class_weight=balanced, clf__estimator__max_depth=10, clf__estimator__n_estimators=2000, features__text_pipeline__tfidf__ngram_range=(1, 2) 


In [53]:
y_better_rfc_pred = cv_better_rfc_model.predict(X_test)

In [54]:
print("\nModel evaluation on secondly tuned RandomForestClassifier with feature selection...")
display_results(TARGET_NAMES, y_test, y_better_rfc_pred, cv_better_rfc_model, better_rfc_param_grid)


Model evaluation on secondly tuned RandomForestClassifier with feature selection...

First: overall accuracy score: 0.070028


C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.45      0.32      0.37       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.56      0.55      0.55      2146
          medical_help       0.13      0.05      0.07       386
      medical_products       0.06      0.01      0.01       253
     search_and_rescue       0.00      0.00      0.00       142
              security       0.00      0.00      0.00       103
              military       0.00      0.00      0.00       179
           child_alone       0.00      0.00      0.00         0
                 water       0.17      0.02      0.04       332
                  food       0.18      0.05      0.08       553
               shelter       0.06      0.01      0.02       456
              clothing       0.33      0.01      0.02    

**Note**:<br>
For the RandomForestClassier the usage of the feature selection improves the accuracy only a little bit, there are still much categories set to 0.0 and having a low best score and still a lot of false negatives when precision is much higher compared to the associated recall value. Next, we add a <i>Normalizer</i> and will have a look if this improves the feature selection result. Furthermore, we test if the randomized cross validation search is much faster.

In [64]:
# try it again with added Normalizer in pipeline and RandomizedSearchCV
print("\n----- RandomForestClassifier with feature selection, Normalizer, better param grid and RandomizedSearchCV -----")
print("Build best model: ...")
randomcv_better_rfc_model = build_model(RandomForestClassifier(n_jobs=1, random_state=FIXED_SEED),
                                        better_rfc_param_grid)
print("Train model: ...")
randomcv_better_rfc_model.fit(X_train, y_train)


----- RandomForestClassifier with feature selection, Normalizer, better param grid and RandomizedSearchCV -----
Build best model: ...
Train model: ...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced 
[CV]  features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced, total=216.2min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 216.2min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced, total=165.6min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced 
[CV]  features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced, total=160.5min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced 
[CV]  features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_weight=balanced, total=158.0min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 2), clf__estimator__n_estimators=2000, clf__estimator__max_depth=10, clf__estimator__class_wei

[CV]  features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_weight=balanced, total=195.5min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_weight=balanced 
[CV]  features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_weight=balanced, total=204.4min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_weight=balanced 
[CV]  features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_weight=balanced, total=214.6min
[CV] features__text_pipeline__tfidf__ngram_range=(1, 3), clf__estimator__n_estimators=2000, clf__estimator__max_depth=20, clf__estimator__class_wei

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 9501.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('text_pipeline',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('tfidf',
                                                                                               TfidfVectorizer(analyzer='word',
                                                                                                               binary=False,
                                                                                                               decode_error='strict',
                                                                           

In [65]:
randomy_better_rfc_pred = randomcv_better_rfc_model.predict(X_test)

In [66]:
print("\nModel evaluation on third tuned RandomForestClassifier with feature selection, normalizer and RandomizedSearch ...")
display_results(TARGET_NAMES, y_test, randomy_better_rfc_pred, randomcv_better_rfc_model, better_rfc_param_grid)


Model evaluation on third tuned RandomForestClassifier with feature selection, normalizer and RandomizedSearch ...

First: overall accuracy score: 0.057296


C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification Report for each target class:
                         precision    recall  f1-score   support

               related       1.00      1.00      1.00      3927
               request       0.29      0.25      0.27       875
                 offer       0.00      0.00      0.00        29
           aid_related       0.56      0.57      0.56      2146
          medical_help       0.10      0.13      0.11       386
      medical_products       0.06      0.07      0.06       253
     search_and_rescue       0.02      0.02      0.02       142
              security       0.04      0.04      0.04       103
              military       0.07      0.09      0.08       179
           child_alone       0.00      0.00      0.00         0
                 water       0.10      0.08      0.09       332
                  food       0.16      0.16      0.16       553
               shelter       0.11      0.09      0.10       456
              clothing       0.03      0.02      0.03    

**Note**:<br>
This looks much better reagarding the feature selection. The decomposition results fits to the ETL pipelines disaster category distribution diagram. So, we will do this calculation again with the <i>GridSearchCV</i> instance and the given model types.

### 9. Export your model as a pickle file

Finally, having found the best model from our model selection list, we save this model with its best parameters as a pickle file. Pickle is the standard way of serialising objects in Python. With this pickle file we can deserialise our model and use it to make new predictions.

In [55]:
def save_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, "wb" ))

In [56]:
# see train_classifier.py file
model_filepath = "classifier.pkl"
model = cv_better_rfc_model
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Best trained model saved!')

Saving model...
    MODEL: classifier.p
Best trained model saved!


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.